In [175]:
import re    # for regular expressions
import nltk  # for text manipulation
import string
import warnings
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

## Load Data

In [176]:
df  = pd.read_csv('tweets.csv')
print(df.shape)

(17216, 7)


In [177]:
df.head(5)

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu,DonaldTrump
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA",DonaldTrump
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz,DonaldTrump


## Data Preprocessing

In [178]:
# Select features
data = df[['text']]
print('Feature selected DataFrame:')
data

Feature selected DataFrame:


,text
0,Here is my statement.pic.twitter.com/WAZiGoQqMQ
1,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu
2,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA"
3,Certainly has been an interesting 24 hours!
4,Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz
...,...
17211,"""My persona will never be that of a wallflower - I’d rather build walls than cling to them"" --Donald J. Trump"
17212,New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e
17213,Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!
17214,Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!


In [179]:
data['ori_tweet'] = data['text']
data

,text,ori_tweet
0,Here is my statement.pic.twitter.com/WAZiGoQqMQ,Here is my statement.pic.twitter.com/WAZiGoQqMQ
1,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu
2,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA","The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA"
3,Certainly has been an interesting 24 hours!,Certainly has been an interesting 24 hours!
4,Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz,Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz
...,...,...
17211,"""My persona will never be that of a wallflower - I’d rather build walls than cling to them"" --Donald J. Trump","""My persona will never be that of a wallflower - I’d rather build walls than cling to them"" --Donald J. Trump"
17212,New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e,New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e
17213,Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!,Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!
17214,Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!,Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!


In [180]:
import re
content=[]
for tweet in data["text"]:
    content.append(re.sub('\@\S+','',tweet))

In [181]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        #Removing link
        url_pattern = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?'
        total_text = re.sub(url_pattern, ' ', total_text)
        # replace every special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', total_text)
        # replace multiple spaces with single space
        total_text = re.sub('\s+',' ', total_text)
        total_text=total_text.replace('realdonaldtrump','').replace('donald','').replace('trump','')
        # converting all the chars into lower-case.
        total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from the data
            if not word in stop_words:
                word=(word)
                string += word + " "
        
        data[column][index] = string

for index, row in df.iterrows():
    if type(row['text']) is str:
        preprocessing(row['text'], index, 'text')

In [182]:
data

,text,ori_tweet
0,,Here is my statement.pic.twitter.com/WAZiGoQqMQ
1,really america terrible,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu
2,media establishment want race badly never drop race never let supporters maga,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA"
3,certainly interesting 24 hours,Certainly has been an interesting 24 hours!
4,debate polls look great thank maga,Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz
...,...,...
17211,persona never wallflower rather build walls cling donald j trump,"""My persona will never be that of a wallflower - I’d rather build walls than cling to them"" --Donald J. Trump"
17212,new blog post celebrity apprentice finale lessons learned along way,New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e
17213,donald trump reads top ten financial tips late show david letterman funny,Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!
17214,donald trump appearing view tomorrow morning discuss celebrity apprentice new book think like champion,Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!


In [183]:
data['text'].isnull().value_counts()

False    17216
Name: text, dtype: int64

In [184]:
tweet=list(data['text'])
tweet

['',
 'really america terrible ',
 'media establishment want race badly never drop race never let supporters maga ',
 'certainly interesting 24 hours ',
 'debate polls look great thank maga ',
 'saying clinton campaign anti catholic bigotry ',
 'thank maga ',
 'cincinnati ohio tomorrow night 7 30pm join ohiovotesearly votetrumppence16 tickets ',
 'little pick dishonest media incredible information provided wikileaks dishonest rigged system ',
 'thank florida movement never seen never seen lets get votetrumppence16 11 8 ',
 'foul mouthed sen john mccain begged support primary gave dropped locker room remarks ',
 'disloyal r far difficult crooked hillary come sides know win teach ',
 'exception cheating bernie nom dems always proven far loyal republicans ',
 'nice shackles taken fight america way want ',
 'weak ineffective leader paul ryan bad conference call members went wild disloyalty ',
 'despite winning second debate landslide every poll hard well paul ryan others give zero support 

In [189]:
import gensim
TaggedDocument = gensim.models.doc2vec.TaggedDocument

In [185]:
cut_sentence = data['text']
def X_train(cut_sentence):
    x_train=[]
    for i,text in enumerate(cut_sentence):
        word_list=text.split(' ')
        l = len(word_list)
        word_list[l-1]=word_list[l-1].strip()
        document = TaggedDocument(word_list,tags=[i])
        x_train.append(document)
    return x_train

In [190]:
T = X_train(cut_sentence)
T

[TaggedDocument(words=[''], tags=[0]),
 TaggedDocument(words=['really', 'america', 'terrible', ''], tags=[1]),
 TaggedDocument(words=['media', 'establishment', 'want', 'race', 'badly', 'never', 'drop', 'race', 'never', 'let', 'supporters', 'maga', ''], tags=[2]),
 TaggedDocument(words=['certainly', 'interesting', '24', 'hours', ''], tags=[3]),
 TaggedDocument(words=['debate', 'polls', 'look', 'great', 'thank', 'maga', ''], tags=[4]),
 TaggedDocument(words=['saying', 'clinton', 'campaign', 'anti', 'catholic', 'bigotry', ''], tags=[5]),
 TaggedDocument(words=['thank', 'maga', ''], tags=[6]),
 TaggedDocument(words=['cincinnati', 'ohio', 'tomorrow', 'night', '7', '30pm', 'join', 'ohiovotesearly', 'votetrumppence16', 'tickets', ''], tags=[7]),
 TaggedDocument(words=['little', 'pick', 'dishonest', 'media', 'incredible', 'information', 'provided', 'wikileaks', 'dishonest', 'rigged', 'system', ''], tags=[8]),
 TaggedDocument(words=['thank', 'florida', 'movement', 'never', 'seen', 'never', 'see

## Doc2Vec 

In [191]:
def train(x_train,size=300):
    model = Doc2Vec(x_train,min_count=1,window=3,size=size,sample=1e-3,nagative=5,workers=4)
    model.train(x_train,total_examples=model.corpus_count,epochs=10)
    return model

In [192]:
model_dm=train(T)

In [149]:
T=X_train(b)
T

[TaggedDocument(words=[''], tags=[0]),
 TaggedDocument(words=['really', 'america', 'terrible', ''], tags=[1]),
 TaggedDocument(words=['media', 'establishment', 'want', 'race', 'badly', 'never', 'drop', 'race', 'never', 'let', 'supporters', 'maga', ''], tags=[2]),
 TaggedDocument(words=['certainly', 'interesting', '24', 'hours', ''], tags=[3]),
 TaggedDocument(words=['debate', 'polls', 'look', 'great', 'thank', 'maga', ''], tags=[4]),
 TaggedDocument(words=['saying', 'clinton', 'campaign', 'anti', 'catholic', 'bigotry', ''], tags=[5]),
 TaggedDocument(words=['thank', 'maga', ''], tags=[6]),
 TaggedDocument(words=['cincinnati', 'ohio', 'tomorrow', 'night', '7', '30pm', 'join', 'ohiovotesearly', 'votetrumppence16', 'tickets', ''], tags=[7]),
 TaggedDocument(words=['little', 'pick', 'dishonest', 'media', 'incredible', 'information', 'provided', 'wikileaks', 'dishonest', 'rigged', 'system', ''], tags=[8]),
 TaggedDocument(words=['thank', 'florida', 'movement', 'never', 'seen', 'never', 'see

In [193]:
tweet_ori = list(data['ori_tweet'])
tweet_ori

['Here is my statement.pic.twitter.com/WAZiGoQqMQ',
 'Is this really America? Terrible!pic.twitter.com/WiwC61PIFu',
 'The media and establishment want me out of the race so badly -  I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA',
 'Certainly has been an interesting 24 hours!',
 'Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz',
 'WHAT THEY ARE SAYING ABOUT THE CLINTON CAMPAIGN’S ANTI-CATHOLIC BIGOTRY: \nhttp://bit.ly/2dcbtvkCrooked\xa0',
 'Thank you! #MAGA #AmericaFirstpic.twitter.com/fG313wjlKm',
 'I will be in Cincinnati, Ohio tomorrow night at 7:30pm- join me! #OhioVotesEarly #VoteTrumpPence16 \nTickets: https://www.donaldjtrump.com/schedule/register/cincinnati-oh2/\xa0…pic.twitter.com/XUFuGc4Fg5',
 'Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system!',
 'Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets

In [214]:
data.to_csv('Tweet_Processed.csv')

## Test Sentence (Top 20 similar)

In [194]:
strl=u'thank you very much'
test_text=strl.split(' ')
inferred_vector=model_dm.infer_vector(doc_words=test_text,alpha=0.025,steps=500)

In [195]:
sims=model_dm.docvecs.most_similar([inferred_vector],topn=20)

In [201]:
for count,sim in sims:
    print(count,sim)
    sentence = tweet[count]
    words=' '
    
    for word in sentence:
        words = words + word+''
        
    print(words,sim)

4333 0.7388485670089722
 thank much nice story greatly appreciate  0.7388485670089722
11956 0.7089517116546631
 thanks great comments recent interviews much appreciated  0.7089517116546631
7364 0.7002252340316772
 donaldtrump2016 realdonaldtrump washington country would better shape wow thank caring much  0.7002252340316772
574 0.6982960104942322
 thank roseanne much  0.6982960104942322
9164 0.6940373182296753
 miranda diplo realdonaldtrump much republican brilliant entrepreneur individual thank  0.6940373182296753
12938 0.683610737323761
 wendywilliams thanks nice statement especially wife kids much appreciated  0.683610737323761
1887 0.6794548034667969
 thank much wonderful article robert  0.6794548034667969
9179 0.6760728359222412
 realjohnanthon realdonaldtrump hate africa much tremendous potential  0.6760728359222412
5880 0.6677308082580566
 toneantone96 realdonaldtrump reason love much compassionate  0.6677308082580566
344 0.667113184928894
 hate say republican convention far int

In [202]:
for count,sim in sims:
    print(count,sim)
    sentence = tweet_ori[count] #original sentences
    words=' '
    
    for word in sentence:
        words = words + word+''
        
    print(words,sim)

4333 0.7388485670089722
 Thank you very much for the nice story, I greatly appreciate it-http://dmreg.co/1IDa9KR  0.7388485670089722
11956 0.7089517116546631
 Thanks for all the great comments on all my recent interviews.  Much appreciated. 0.7089517116546631
7364 0.7002252340316772
 "@DonaldTrump2016: @realDonaldTrump if you were in Washington this country would be in better shape"  Wow, thank you for caring so much! 0.7002252340316772
574 0.6982960104942322
 Thank you Roseanne, very much appreciated.http://thehill.com/blogs/blog-briefing-room/news/282864-roseanne-barr-we-would-be-so-lucky-if-trump-won … 0.6982960104942322
9164 0.6940373182296753
 "@Miranda__Diplo: @realDonaldTrump is much more than just a Republican. He is a brilliant entrepreneur and individual." Thank you. 0.6940373182296753
12938 0.683610737323761
 .@WendyWilliams Thanks for the nice statement, especially about my wife and kids- very much appreciated. 0.683610737323761
1887 0.6794548034667969
 Thank you so much fo

## Save Model 

In [203]:
model_dm.save("doc2vec.model")

In [213]:
from sklearn.externals import joblib
# Save model
joblib.dump(model_dm, 'Doc2Vec-Model.pkl')

['Doc2Vec-Model.pkl']

## Load Model 

In [215]:
import pandas as pd
from gensim.models import Doc2Vec
model_test = Doc2Vec.load("doc2vec.model")
data  = pd.read_csv('Tweet_Processed.csv')

In [219]:
#tweet_ori = list(data['ori_tweet'])
tweet=list(data['text'])
strl=u'you are so great'
test_text=strl.split(' ')
inferred_vector=model_test.infer_vector(doc_words=test_text,alpha=0.025,steps=500)
sims=model_test.docvecs.most_similar([inferred_vector],topn=20)

for count,sim in sims:
    print(count,sim)
    sentence = tweet[count]
    words=' '
    
    for word in sentence:
        words = words + word+''
        
    print(words)

9699 0.941011905670166
 justinrose99 great playing proud 
12453 0.9358325004577637
 chuckwoolery billmaher chuck great thanks 
13398 0.9340419769287109
 believable state employment great jack welch government bureaucrat voting obama 
3609 0.9327719211578369
 great 
15335 0.9324244856834412
 michael vick philadelphia eagles great athlete great quarterback 
13410 0.9307888746261597
 congrats yankees finishing 1st al east derek jeter great good luck playoffs 
14530 0.9281915426254272
 behind scemes century21 super bowl commercial great time 
2130 0.926156759262085
 mcjeff42 realdonaldtrump made snl great 
12604 0.9257597923278809
 great believer asking everyone opinion make decision natural reflex art deal 
9322 0.9244785904884338
 jamesshelby realdonaldtrump opinion tebow destroyed jets great guy gone jacksonville 
15458 0.9239096641540527
 believable state employment great jack welch government bureaucrat voting obama 
12088 0.9202898740768433
 markrucci e hig great going mark 
4425 0.9

In [222]:
tweet_ori = list(data['ori_tweet'])
strl=u'you are so great'
test_text=strl.split(' ')
inferred_vector=model_test.infer_vector(doc_words=test_text,alpha=0.025,steps=500)
sims=model_test.docvecs.most_similar([inferred_vector],topn=20)

for count,sim in sims:
    print(count,sim)
    sentence = tweet_ori[count]
    words=' '
    
    for word in sentence:
        words = words + word+''
        
    print(words)

9699 0.9418283700942993
 .@JustinRose99 Great playing--we are proud of you!
3609 0.9378787279129028
 Great, thanks.https://twitter.com/isaacbeisbol507/status/609345354564272128 …
15335 0.9349060654640198
 Michael Vick of the Philadelphia @eagles is a great athlete but not a great quarterback.
13398 0.9338043332099915
 Who is more believable on the state of employment--the great @jack_welch or some government bureaucrat who is voting for Obama?
13410 0.9336014986038208
 Congrats to @Yankees on finishing 1st in the AL East. Derek Jeter is great--good luck in the playoffs!
12453 0.9335277080535889
 @chuckwoolery @billmaher Chuck--you are great--thanks!
14530 0.9269655346870422
 Behind the scemes of my @CENTURY21 Super Bowl commercial http://youtu.be/Y6HWdR_11EI   Had a great time.
9322 0.9238348007202148
 "@jamesshelby: @realDonaldTrump What is your opinion on Tebow?" Was destroyed at Jets-GREAT GUY-should have gone to Jacksonville!
12604 0.9232172966003418
 “I’m a great believer in askin